In [1]:
# -*- coding: utf-8 -*-
import os
import tensorflow as tf
%run Bi_LSTM
%run Word2Vec
import gensim
import numpy as np
from eunjeon import Mecab
import pandas as pd

C:\Users\dongc\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
W2V = Word2Vec()
mecab = Mecab()
from konlpy.tag import Okt
okt = Okt()

In [3]:
df = pd.read_csv('../data/자소서_잡코추가2.csv',index_col=0, engine='python', encoding='utf8')

In [4]:
Batch_size = 1
Vector_size = 300
Maxseq_length = 922   ## Max length of training data
learning_rate = 0.001
lstm_units = 128
num_class = len(df.category.unique())
keep_prob = 1.0

array(['경영지원', '생산·물류', '재무·금융', '영업·마케팅', '응용프로그래밍', '인사', '통계',
       '서버·시스템'], dtype=object)

In [5]:
tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape = [None, Maxseq_length, Vector_size], name = 'X')
Y = tf.placeholder(tf.float32, shape = [None, num_class], name = 'Y')
seq_len = tf.placeholder(tf.int32, shape = [None])

BiLSTM = Bi_LSTM(lstm_units, num_class, keep_prob)

with tf.variable_scope("loss", reuse = tf.AUTO_REUSE):
    logits = BiLSTM.logits(X, BiLSTM.W, BiLSTM.b, seq_len)
    loss, optimizer = BiLSTM.model_build(logits, Y, learning_rate)

prediction = tf.nn.softmax(logits)

In [ ]:
saver = tf.train.Saver()
init = tf.global_variables_initializer()
modelName = "Bidirectional_LSTM4/BiLSTM_model.ckpt"

sess = tf.Session()
sess.run(init)
saver.restore(sess, modelName)

In [6]:
def classify(jss):
    tokens = okt.nouns(jss)
    embedding = W2V.Convert2Vec('../data/word2vec_okt_nouns.model', tokens)
    zero_pad = W2V.Zero_padding(embedding, Batch_size, Maxseq_length, Vector_size)
    global sess
    result =  sess.run(tf.argmax(prediction,1), feed_dict = {X: zero_pad , seq_len: [len(tokens)] } ) 
    print(sess.run(prediction, feed_dict = {X: zero_pad , seq_len: [len(tokens)] } ) )
    if result == 0:
        print('경영지원')
    elif result == 1:
        print('재무·금융')
    elif result == 2:
        print('생산·물류')
    elif result == 3:
        print('영업·마케팅')
    elif result == 4:
        print('인사')
    elif result == 5:
        print('서버·시스템')
    elif result == 6:
        print('응용프로그래밍')

ERROR! Session/line number was not unique in database. History logging moved to new session 151


In [7]:
df.groupby(['category']).agg('count')

,c_origin,jss,label
category,,,
경영지원,5352,5352,5352
생산·물류,4457,4457,4457
서버·시스템,1712,1712,1712
영업·마케팅,9803,9803,9803
응용프로그래밍,3130,3130,3130
인사,741,741,741
재무·금융,3350,3350,3350
통계,225,225,225


In [19]:
df[df.category=='영업·마케팅'].jss[3591]

"[똑 부러지는 똑순이]\n어렸을 때 친척들과 동네 어른 분들께서는 저를 '똑순이'라고 부르셨습니다. 책임감 있게 일을 하는 모습이나, 하고자 하는 일은 계획적으로 똑 부러지게 해내는 모습을 보고 붙여주신 별명이었습니다. 장녀로 자라오며 일찌감치 자립심을 가질 수 있었고, 야무지고 책임감 강한 아이로 성장하였습니다.\n\n[완벽한 화음을 위해]\n동호회 등 다양한 모임을 통해 경험과 지식을 쌓는 것은 물론, 즐거움이라는 가치 또한 발견했습니다. 그런 즐거움을 바탕으로 팀 내에서 '기쁨조' 역할을 도맡아, 활기 넘치는 팀 분위기가 형성될 수 있도록 하였습니다. 또한, 밝은 인상에서 나오는 편안함 덕분에 주위 사람들에게 친근한 이미지로 다가설 수 있었습니다.\n\n[똑순이+기쁨조=봉이 김선달]\n아무리 좋은 상품을 기획하고, 우수한 공정을 걸쳐 불량률이 없게 생산을 한다고 하더라도, 제대로 된 판매 능력이 없다면 기업에게는 아무런 가치가 없습니다. 아무런 가치가 없는 대동강 물을 팔았다는 ‘봉이 김선달’은 영업인으로는 정말 대단한 일을 해낸 것이며 그 역할 모델로 삼아야 할 사람입니다. \n영업에 있어, 주위를 밝게 만들어 웃음으로 만드는 성격과 더불어 똑부러진 성격으로 원가 및 판매가를 맞춰 상품을 판매하는 점이 저에게는 최대의 강점이라고 생각합니다. ‘봉이 김선달’같은 존재로 노루페인트에서 최고의 영업인이 되려 합니다."

In [9]:
for category1 in df.category1.unique():
    print(category1)
    print(df[df.category1 == category1].iloc[0].jss)
    print('\n\n\n\n\n')

경영기획
[제품 홍보가 곧 기업 홍보다]
제품 홍보가 곧 기업 홍보라는 회사의 모토처럼, 학창시절부터 즐겨먹어 온 맛있는 우유 GT를 통해 남양유업을 알았고, 맛있고 신선한 우유를 만드는 기업이라는 이미지를 가지고 있었습니다. 그렇게 남양유업에 우호적인 관심을 가지고 있다가, 새로운 성장을 위한 전략으로 해외 수출을 시작한 남양유업의 혁신적인 경영방침에 빠져들게 되었습니다. 선택과 집중의 전략으로 ‘세계적 유제품 전문 회사’로 거듭날 것을 믿기에, 남양유업 인으로의 제 모든 역량과 혼신을 쏟아 붇고자 지원합니다. 다양한 도전을 통해 연마한 창조적 사고와 주어진 일에 제 자신을 불태울 수 있는 열정, 맡은 일에 대한 책임감과 성실함을 바탕으로 경영관리부서에서 일함으로써 남양유업의 발전에 동참할 수 있는 기회를 갖고 싶습니다. 그러기 위해 제가 맡은 분야에서 최고가 되겠다는 목표를 가지고서 현재에 안주하지 않고 미래를 위한 자기계발에 노력할 것입니다






생산품질
[열정은 신뢰다]
"열정을 다하는 사람에게는 신뢰가 따라온다." 아버지께서 저에게 항상 하시던 말씀입니다. 주변 업체들과의 경쟁 속에서도 거래처와의 끈끈한 관계를 유지하시며, 하나의 제품에도 최선을 다하시는 아버지의 모습은 저에게 큰 영향을 주었습니다. 그래서 저는 맡은 일에 대해 열정을 쏟으며, 신뢰받는 사람이 되고자 합니다. 

[성공=실천+경험]
목표를 성취함에 있어, 가장 중요한 것은 경험입니다. 전공 시험 중에 차세대 디스플레이와 관련된 문제가 나왔습니다. 처음에는 정답이 잘 기억이 나지 않아서 당황했습니다. 하지만 신문에서 봤던 차세대 디스플레이에 대하여 글을 썼고, 결국 정답으로 처리되면서 A+을 받을 수 있었습니다






금융
관찰력이 좋은 것이 장점입니다.
사람들의 일상에 관심이 많습니다. 이따금씩 아파트 베란다를 통해 옆동의 사람들을 보면, 참 신기하게 느껴지는 것들이 있습니다. 모두 같은 공간 속에서 다르게 행동하는데, 그 안의 전체적인 일상이 굉장히 비슷하다는 것을 보게 됩

In [ ]:
while(1):
    s = input("문장을 입력하세요 : ")
    if s == '':
        break
    else:
        classify(s)

문장을 입력하세요 : [똑 부러지는 똑순이]\n어렸을 때 친척들과 동네 어른 분들께서는 저를 '똑순이'라고 부르셨습니다. 책임감 있게 일을 하는 모습이나, 하고자 하는 일은 계획적으로 똑 부러지게 해내는 모습을 보고 붙여주신 별명이었습니다. 장녀로 자라오며 일찌감치 자립심을 가질 수 있었고, 야무지고 책임감 강한 아이로 성장하였습니다.\n\n[완벽한 화음을 위해]\n동호회 등 다양한 모임을 통해 경험과 지식을 쌓는 것은 물론, 즐거움이라는 가치 또한 발견했습니다. 그런 즐거움을 바탕으로 팀 내에서 '기쁨조' 역할을 도맡아, 활기 넘치는 팀 분위기가 형성될 수 있도록 하였습니다. 또한, 밝은 인상에서 나오는 편안함 덕분에 주위 사람들에게 친근한 이미지로 다가설 수 있었습니다.\n\n[똑순이+기쁨조=봉이 김선달]\n아무리 좋은 상품을 기획하고, 우수한 공정을 걸쳐 불량률이 없게 생산을 한다고 하더라도, 제대로 된 판매 능력이 없다면 기업에게는 아무런 가치가 없습니다. 아무런 가치가 없는 대동강 물을 팔았다는 ‘봉이 김선달’은 영업인으로는 정말 대단한 일을 해낸 것이며 그 역할 모델로 삼아야 할 사람입니다. \n영업에 있어, 주위를 밝게 만들어 웃음으로 만드는 성격과 더불어 똑부러진 성격으로 원가 및 판매가를 맞춰 상품을 판매하는 점이 저에게는 최대의 강점이라고 생각합니다. ‘봉이 김선달’같은 존재로 노루페인트에서 최고의 영업인이 되려 합니다
[[0.11735515 0.1226902  0.12453378 0.13297407 0.12171268 0.13583846
  0.13228957 0.1126061 ]]
서버·시스템
